In [1]:
import pandas as pd
from eth_abi import abi
import eth_utils
import pickle as pkl
import os
import time

from sqlalchemy import create_engine

# Import sqlalchemy types
from sqlalchemy import MetaData, Table, Column, Integer, String, DateTime, Float, Boolean, BigInteger, LargeBinary

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Adjust the minimum with of a pandas column
pd.set_option('display.max_colwidth', None)

In [3]:
postgresql_uri_us = os.environ["POSTGRESQL_URI_US"]
postgresql_uri_mp = os.environ["POSTGRESQL_URI_MP"]

## Decode the data from the binary format

In [4]:

search_commands = ['00', '01']
command_mapping = {"00": "V3_SWAP_EXACT_IN", 
                   "01": "V3_SWAP_EXACT_OUT"}
'''
select *
from uniswap_ethereum.UniversalRouter_call_execute
limit 1000
'''
df = pd.read_csv("data.csv", low_memory=False)

if not os.path.exists("clean"):
    os.mkdir("clean")

df

,contract_address,call_success,call_tx_hash,call_trace_address,call_block_time,call_block_number,commands,deadline,inputs
0,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,True,0x26efe3924b80edeb439a6cd008e4efc300cd16735fa9a39fb159342d4c4d5e19,[],2023-08-07 15:42:23.000 UTC,17864015.0,0x080c,1.691425e+09,[0x0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000de0b6b3a764000000000000000000000000000000000000000000000000000000135290b044b822800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000002000000000000000000000000dc5e9445169c73cf21e1da0b270e8433cac69959000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0x00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000135290b044b8228]
1,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,True,0x9b234583f8af9aa11666f4f8362eb597dfe19e5e4d502a0bffae733cfb501ec6,[],2023-08-07 15:42:23.000 UTC,17864015.0,0x0b08,1.691425e+09,[0x0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000016345785d8a0000 0x0000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000016345785d8a0000000000000000000000000000000000000000000000000010806d9810a479f9af00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc200000000000000000000000038029c62dfa30d9fd3cadf4c64e9b2ab21dbda17]
2,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,True,0x3adb1878eb4e390ca354a087dde15f884557d11858e65ca0081965e0946fb793,[],2023-08-07 15:42:23.000 UTC,17864015.0,0x080c,1.691425e+09,[0x000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000001b091447e9c70000000000000000000000000000000000000000000000000818376b03c3222b00000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000020000000000000000000000002577944fd4b556a99cc5aa0f072e4b944aa088df000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0x00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000818376b03c3222b]
3,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,True,0x00bc709fb31a095444b3277c6499978a6b2cfa5b324cbee941f6701a2329112d,[],2023-08-07 15:42:23.000 UTC,17864015.0,0x08,1.691425e+09,[0x0000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000009184e72a00000000000000000000000000000000000000000000000000042587784b50fd1d400000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000200000000000000000000000072e4f9f808c49a2a61de9c5896298920dc4eeea9000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2]
4,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,True,0x868a4da8d302128e1614d620e68925927e55ca5672e1a7dfffadbd53501ce224,[],2023-08-07 15:42:23.000 UTC,17864015.0,0x0b00,1.691425e+09,[0x000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000002f129cd2c705ee6 0x000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000002f129cd2c705ee60000000000000000000000000000000000000000006d4fc8c3a0f1d2ea257a4200000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002bc02aaa

In [5]:
# vectorized clean the data
df['split_command'] = df['commands'].str.rstrip().str.split("").apply(lambda x: [_x for _x in x if _x != ""])
df['clean_commands'] = df['split_command'].apply(lambda x: [x[i] + x[i+1] for i in range(0, len(x), 2)])
df['clean_commands'] = df['clean_commands'].apply(lambda x: [_x for _x in x if _x != "0x"]).copy()
df['command_location'] = df['clean_commands'].apply(lambda x: [count for (count, row) in enumerate(x) 
                                                               if row in search_commands])

df['clean_inputs'] = df['inputs'].apply(lambda x: [_x.replace("[", "").replace("]", "") for _x in x.split(" ")])

v3_swaps = df[df['command_location'].apply(lambda x: len(x) != 0)].copy()

v3_swaps

,contract_address,call_success,call_tx_hash,call_trace_address,call_block_time,call_block_number,commands,deadline,inputs,split_command,clean_commands,command_location,clean_inputs
4,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,True,0x868a4da8d302128e1614d620e68925927e55ca5672e1a7dfffadbd53501ce224,[],2023-08-07 15:42:23.000 UTC,17864015.0,0x0b00,1.691425e+09,[0x000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000002f129cd2c705ee6 0x000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000002f129cd2c705ee60000000000000000000000000000000000000000006d4fc8c3a0f1d2ea257a4200000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002bc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2002710423f4e6138e475d85cf7ea071ac92097ed631eea000000000000000000000000000000000000000000],"[0, x, 0, b, 0, 0]","[0b, 00]",[1],"[0x000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000002f129cd2c705ee6, 0x000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000002f129cd2c705ee60000000000000000000000000000000000000000006d4fc8c3a0f1d2ea257a4200000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002bc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2002710423f4e6138e475d85cf7ea071ac92097ed631eea000000000000000000000000000000000000000000]"
9,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,True,0x9cce38269a1b337084c4c31b73bac9bc75ae565b0ad0f43169605f81121ab28c,[],2023-08-07 15:42:23.000 UTC,17864015.0,0x0b010c,1.691425e+09,[0x00000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000002a3d2d2a50b0733d 0x00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000147d357000000000000000000000000000000000000000000000000002a3d2d2a50b0733d00000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002ba0b86991c6218b36c1d19d4a2e9eb0ce3606eb48000bb8c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2000000000000000000000000000000000000000000 0x00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000],"[0, x, 0, b, 0, 1, 0, c]","[0b, 01, 0c]",[1],"[0x00000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000002a3d2d2a50b0733d, 0x00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000147d357000000000000000000000000000000000000000000000000002a3d2d2a50b0733d00000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002ba0b86991c6218b36c1d19d4a2e9eb0ce3606eb48000bb8c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2000000000000000000000000000000000000000000, 0x00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000]"
11,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,True,0x0815280508f43c94e4a0a2991f24108124effac762c120022b9e66febbe88696,[],2023-08-07 15:42:23.000 UTC,17864015.0,0x0a000c,1.691423e+09,[0x000000000000000000000000dac17f958d2ee523a2206206994597c13d831ec7000000000000000000000000ffffffffffffffffffffffffffffffffffffffff0000000000000000000000000000000000000000000000000000000064f89db500000000000000000000000000000000000000000000000000000000000000000000000000000000000000003fc91a3afd70395cd496c647d5a6cc9d4b2b7fad00000000000

In [6]:
all_data = []
for i in range(v3_swaps.shape[0]):
    swap = v3_swaps.iloc[i]
    # swap_payload = []

    for command_location in swap['command_location']:
        
        command = swap['clean_commands'][command_location]
        input_str = swap['clean_inputs'][command_location]
        input_str = eth_utils.decode_hex(input_str)

        decodedABI = abi.decode(['address', 'uint256', 'uint256', 'bytes', 'bool'], input_str)

        path = decodedABI[3]

        payload = []
        offset = 43

        addr1 = eth_utils.to_hex(path[0:20])
        fee = eth_utils.to_int(path[20:23])
        addr2 = eth_utils.to_hex(path[23:43])
        payload.append([addr1, fee, addr2])

        while offset < len(path):
            # consume the path
            addr1 = addr2
            fee = eth_utils.to_int(path[offset: offset+3])
            addr2 = eth_utils.to_hex(path[offset+3: offset+23])

            payload.append([addr1, fee, addr2])

            offset += 23

        if len(path) < 43:
            # this can only happen if bug (i think)
            raise ValueError("Incorrect Path Specification")
        
        command_str = command_mapping[command]
        cleaned_input = {"recipient": decodedABI[0],
                    f"{'amountIn' if command == '00' else 'amountOut'}": decodedABI[1],
                    f"{'amountOutMin' if command == '00' else 'amountInMax'}": decodedABI[2],
                    "path": payload,
                    "payerIsUser": decodedABI[4]}
        
        # swap_payload.append([command_str, cleaned_input, swap['call_tx_hash']])
        all_data.append([command_str, cleaned_input, swap['call_tx_hash']])
    # all_data.append(swap_payload)


cleaned_data = [item for item in all_data if len(item[1]["path"]) == 1]


for item in cleaned_data[:10]:
    print(item)

len(cleaned_data)

['V3_SWAP_EXACT_IN', {'recipient': '0x0000000000000000000000000000000000000001', 'amountIn': 211996618653720294, 'amountOutMin': 132149684735488745768581698, 'path': [['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 10000, '0x423f4e6138e475d85cf7ea071ac92097ed631eea']], 'payerIsUser': False}, '0x868a4da8d302128e1614d620e68925927e55ca5672e1a7dfffadbd53501ce224']
['V3_SWAP_EXACT_OUT', {'recipient': '0x0000000000000000000000000000000000000001', 'amountOut': 5500000000, 'amountInMax': 3043638582937940797, 'path': [['0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 3000, '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']], 'payerIsUser': False}, '0x9cce38269a1b337084c4c31b73bac9bc75ae565b0ad0f43169605f81121ab28c']
['V3_SWAP_EXACT_IN', {'recipient': '0x0000000000000000000000000000000000000002', 'amountIn': 5273000000, 'amountOutMin': 2882045483002369817, 'path': [['0xdac17f958d2ee523a2206206994597c13d831ec7', 500, '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']], 'payerIsUser': True}, '0x0815280508f43c94

46804

In [7]:
flattened_data = []
for item in cleaned_data:
    transaction_type = item[0]
    transaction_details = item[1]
    transaction_hash = item[2]

    # Flatten path separately as it's a nested list
    paths = transaction_details['path']
    for path in paths:
        token0, fee, token1 = path
        flattened_data.append({
            'transaction_type': transaction_type,
            'transaction_hash': transaction_hash,
            'recipient': transaction_details['recipient'],
            'amountIn': transaction_details.get('amountIn'),
            'amountOut': transaction_details.get('amountOut'),
            'amountOutMin': transaction_details.get('amountOutMin'),
            'amountInMax': transaction_details.get('amountInMax'),
            'payerIsUser': transaction_details['payerIsUser'],
            'token0': token0,
            'fee': fee,
            'token1': token1,
        })

# Convert to DataFrame
df = pd.DataFrame(flattened_data, dtype='object')
df

,transaction_type,transaction_hash,recipient,amountIn,amountOut,amountOutMin,amountInMax,payerIsUser,token0,fee,token1
0,V3_SWAP_EXACT_IN,0x868a4da8d302128e1614d620e68925927e55ca5672e1a7dfffadbd53501ce224,0x0000000000000000000000000000000000000001,211996618653720294,None,132149684735488745768581698,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000,0x423f4e6138e475d85cf7ea071ac92097ed631eea
1,V3_SWAP_EXACT_OUT,0x9cce38269a1b337084c4c31b73bac9bc75ae565b0ad0f43169605f81121ab28c,0x0000000000000000000000000000000000000001,None,5500000000,None,3043638582937940797,False,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2,V3_SWAP_EXACT_IN,0x0815280508f43c94e4a0a2991f24108124effac762c120022b9e66febbe88696,0x0000000000000000000000000000000000000002,5273000000,None,2882045483002369817,None,True,0xdac17f958d2ee523a2206206994597c13d831ec7,500,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
3,V3_SWAP_EXACT_IN,0x326e3cacb000b3dc63578710e619a268a2734373c75ecd1df31c4a259789b4d1,0x0000000000000000000000000000000000000002,200000000000000000000000,None,850173983075597316,None,True,0x0c572544a4ee47904d54aaa6a970af96b6f00e1b,10000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
4,V3_SWAP_EXACT_IN,0x707c06a31c4c8f91b6c142fb11ea685cfcdca781cdd79bce9298ceab02a3fb9d,0x0000000000000000000000000000000000000001,700000000000000000,None,433907060627622534592161845,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000,0x423f4e6138e475d85cf7ea071ac92097ed631eea
...,...,...,...,...,...,...,...,...,...,...,...
46799,V3_SWAP_EXACT_IN,0xd84d050053a8e2c9bfe52dedb3a26acb50008bf8cb4890c839de3d00d6f96457,0x0000000000000000000000000000000000000002,17784178027431205337358015240,None,38588150927773625,None,True,0x3d806324b6df5af3c1a81acba14a8a62fe6d643f,10000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
46800,V3_SWAP_EXACT_IN,0xdcfc62024e8a292defd1b9a8911a4c13118056cbd532b7bb54b813b97d4e241f,0x0000000000000000000000000000000000000002,127324845200000000000,None,114400918410618210,None,True,0x9813037ee2218799597d83d4a5b6f3b6778218d9,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
46801,V3_SWAP_EXACT_OUT,0xca2f17d14e8b8d6b80ca5ac6a8937d5791a706419701113bfb571d702ea069fe,0x0000000000000000000000000000000000000001,None,497000000,None,274024142686152539,False,0xdac17f958d2ee523a2206206994597c13d831ec7,500,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
46802,V3_SWAP_EXACT_IN,0xfc96750af5a8c3836ae1fbb5b2d103f4ce6771509d1b625cd67df7a618ff1ce6,0x0000000000000000000000000000000000000002,25584384046563895003144,None,46693115659367729,None,True,0x55296f69f40ea6d20e478533c15a6b08b654e758,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2


## Get the initial set of transactions to focus on

In [8]:
df = df[(~(df.amountOutMin == 0))]
df = df[(~(df.amountInMax == 0))]

df

,transaction_type,transaction_hash,recipient,amountIn,amountOut,amountOutMin,amountInMax,payerIsUser,token0,fee,token1
0,V3_SWAP_EXACT_IN,0x868a4da8d302128e1614d620e68925927e55ca5672e1a7dfffadbd53501ce224,0x0000000000000000000000000000000000000001,211996618653720294,None,132149684735488745768581698,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000,0x423f4e6138e475d85cf7ea071ac92097ed631eea
1,V3_SWAP_EXACT_OUT,0x9cce38269a1b337084c4c31b73bac9bc75ae565b0ad0f43169605f81121ab28c,0x0000000000000000000000000000000000000001,None,5500000000,None,3043638582937940797,False,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2,V3_SWAP_EXACT_IN,0x0815280508f43c94e4a0a2991f24108124effac762c120022b9e66febbe88696,0x0000000000000000000000000000000000000002,5273000000,None,2882045483002369817,None,True,0xdac17f958d2ee523a2206206994597c13d831ec7,500,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
3,V3_SWAP_EXACT_IN,0x326e3cacb000b3dc63578710e619a268a2734373c75ecd1df31c4a259789b4d1,0x0000000000000000000000000000000000000002,200000000000000000000000,None,850173983075597316,None,True,0x0c572544a4ee47904d54aaa6a970af96b6f00e1b,10000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
4,V3_SWAP_EXACT_IN,0x707c06a31c4c8f91b6c142fb11ea685cfcdca781cdd79bce9298ceab02a3fb9d,0x0000000000000000000000000000000000000001,700000000000000000,None,433907060627622534592161845,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10000,0x423f4e6138e475d85cf7ea071ac92097ed631eea
...,...,...,...,...,...,...,...,...,...,...,...
46799,V3_SWAP_EXACT_IN,0xd84d050053a8e2c9bfe52dedb3a26acb50008bf8cb4890c839de3d00d6f96457,0x0000000000000000000000000000000000000002,17784178027431205337358015240,None,38588150927773625,None,True,0x3d806324b6df5af3c1a81acba14a8a62fe6d643f,10000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
46800,V3_SWAP_EXACT_IN,0xdcfc62024e8a292defd1b9a8911a4c13118056cbd532b7bb54b813b97d4e241f,0x0000000000000000000000000000000000000002,127324845200000000000,None,114400918410618210,None,True,0x9813037ee2218799597d83d4a5b6f3b6778218d9,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
46801,V3_SWAP_EXACT_OUT,0xca2f17d14e8b8d6b80ca5ac6a8937d5791a706419701113bfb571d702ea069fe,0x0000000000000000000000000000000000000001,None,497000000,None,274024142686152539,False,0xdac17f958d2ee523a2206206994597c13d831ec7,500,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
46802,V3_SWAP_EXACT_IN,0xfc96750af5a8c3836ae1fbb5b2d103f4ce6771509d1b625cd67df7a618ff1ce6,0x0000000000000000000000000000000000000002,25584384046563895003144,None,46693115659367729,None,True,0x55296f69f40ea6d20e478533c15a6b08b654e758,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2


## Put the transactions into the database

In [9]:
engine = create_engine(postgresql_uri_us)

In [10]:
pools = pd.read_sql_query(
    """
    SELECT * FROM factory
    """,
    engine,
)[['token0', 'token1', 'fee', "pool"]]

pools

,token0,token1,fee,pool
0,0x8d96b4ab6c741a4c8679ae323a100d74f085ba8f,0xdac17f958d2ee523a2206206994597c13d831ec7,100,0x5c2a6a370e63b4de95650edc565ad0fbab5312ae
1,0x4ad7a056191f4c9519facd6d75fa94ca26003ace,0xdac17f958d2ee523a2206206994597c13d831ec7,100,0x554b887c98fcfac8b3acb9e40a2ab723dc2d8370
2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xf1182229b71e79e504b1d2bf076c15a277311e05,100,0x996dae6bad0ce9eb1f4924953129429699d3c584
3,0x8eee1a04b3223d1667152f5fb148f6ea6420d755,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,100,0x1813316975d62fcd72e7beaff93d8c6e7f52cb8f
4,0x67639ca498a180b3933d01c806bc5cb49f7154d8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,100,0x78a1b525730cc939d59147221ee61924997a5be5
...,...,...,...,...
14336,0x5b7e436cc7652c32e327e62293350b212fb46f7c,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0x9bbbfbf3d6e3ef36550fee8fc61f909c9e1b43fc
14337,0x809e427fad772042e2507a144634aecbd559f7f1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0xa3524cc509338fff7ffa684b210ef8a03ad771de
14338,0x9c4f48122176c39fa2a57aebed337b7874f0266a,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0x05117772aee309f389d23f712fe4e135f3c0c0a0
14339,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xeb1c06e85f2c4c7115b87d57bf014c1dead06146,3000,0xc51cdedf2ab2ac5113f1c3de0876850278c625bc


In [11]:
join_key_func = lambda x: f"{x['token0']}-{x['token1']}-{x['fee']}" if x['token0'] <= x['token1'] else f"{x['token1']}-{x['token0']}-{x['fee']}"

pools = pools.assign(join_key=pools.apply(join_key_func, axis=1)).drop(columns=['token0', 'token1', 'fee'])

pools

,pool,join_key
0,0x5c2a6a370e63b4de95650edc565ad0fbab5312ae,0x8d96b4ab6c741a4c8679ae323a100d74f085ba8f-0xdac17f958d2ee523a2206206994597c13d831ec7-100
1,0x554b887c98fcfac8b3acb9e40a2ab723dc2d8370,0x4ad7a056191f4c9519facd6d75fa94ca26003ace-0xdac17f958d2ee523a2206206994597c13d831ec7-100
2,0x996dae6bad0ce9eb1f4924953129429699d3c584,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48-0xf1182229b71e79e504b1d2bf076c15a277311e05-100
3,0x1813316975d62fcd72e7beaff93d8c6e7f52cb8f,0x8eee1a04b3223d1667152f5fb148f6ea6420d755-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-100
4,0x78a1b525730cc939d59147221ee61924997a5be5,0x67639ca498a180b3933d01c806bc5cb49f7154d8-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-100
...,...,...
14336,0x9bbbfbf3d6e3ef36550fee8fc61f909c9e1b43fc,0x5b7e436cc7652c32e327e62293350b212fb46f7c-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-3000
14337,0xa3524cc509338fff7ffa684b210ef8a03ad771de,0x809e427fad772042e2507a144634aecbd559f7f1-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-3000
14338,0x05117772aee309f389d23f712fe4e135f3c0c0a0,0x9c4f48122176c39fa2a57aebed337b7874f0266a-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-3000
14339,0xc51cdedf2ab2ac5113f1c3de0876850278c625bc,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-0xeb1c06e85f2c4c7115b87d57bf014c1dead06146-3000


In [14]:
df = df.assign(join_key=df.apply(join_key_func, axis=1))

df_joined = df.merge(pools, on='join_key', how='inner').drop(columns=['join_key'])

df_joined

,transaction_type,transaction_hash,recipient,amountIn,amountOut,amountOutMin,amountInMax,payerIsUser,token0,fee,token1,pool
0,V3_SWAP_EXACT_OUT,0x9cce38269a1b337084c4c31b73bac9bc75ae565b0ad0f43169605f81121ab28c,0x0000000000000000000000000000000000000001,None,5500000000,None,3043638582937940797,False,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8
1,V3_SWAP_EXACT_IN,0x15999fd52f3ed1727ec190f6fb4673450b2e0b62f199c7d1eceeca301d946182,0x0000000000000000000000000000000000000001,650000000000000000,None,1103352830,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8
2,V3_SWAP_EXACT_IN,0x7222ecbbb6dd33521c05d478880c61ee3a4296c631b28c71f9d5a7c7111c3f34,0x0000000000000000000000000000000000000001,10000000000000000000,None,18140883228,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8
3,V3_SWAP_EXACT_IN,0x2dd12bd8609a3bf9e302cc56e2e985922c10dca666b9a31946db672f7fdf0e54,0x0000000000000000000000000000000000000001,2200000000000000000,None,3784056538,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8
4,V3_SWAP_EXACT_IN,0x52d534f83c27061d2b8889c9f3068f650f3dda2bd1ba4c41fc923bc6bfa747c2,0x0000000000000000000000000000000000000001,3533224512882063209,None,6409780921,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8
...,...,...,...,...,...,...,...,...,...,...,...,...
18870,V3_SWAP_EXACT_IN,0xbe2d6d7bc0d29907981f6222bdc860170e6d54332b5fdea3735f866caa23eab2,0x0000000000000000000000000000000000000002,351880940000000000000000,None,1026192982190176822,None,True,0x2a2550e0a75acec6d811ae3930732f7f3ad67588,10000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x5e97da599e8c72afea37024240bfca886fb8bc96
18871,V3_SWAP_EXACT_IN,0xaf0949be9fbd4268d7bb28322ef46c40911ee352aaafd77592a2d7c7a3deaa41,0x0000000000000000000000000000000000000002,14186295104880139128246102,None,472650797817056376,None,True,0x8cf8e9e63c3f39eb97a1e8020397bda93cc07196,10000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x37dfe3f7c42bd62509d6d31a53aed18805a34369
18872,V3_SWAP_EXACT_IN,0x9813cb62b75b03395c67ff919277825581b10fd0dcb877a55c72d34202a60089,0x0000000000000000000000000000000000000001,2500000000000000,None,417998905226446063984,None,False,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3000,0x001a8ffcb0f03e99141652ebcdecdb0384e3bd6c,0xf766436b551d2acb09b73d126fd49869541dfa26
18873,V3_SWAP_EXACT_IN,0xf4c22504de2e141335421b728ff015bf11c9f3b677e32d53bfbce32d8eb5a980,0x0000000000000000000000000000000000000002,836623000000000000000000,None,68460438323977550,None,True,0x24ae124c4cc33d6791f8e8b63520ed7107ac8b3e,3000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xd2ef54450ec52347bde3dab7b086bf2a005601d8


In [15]:
df_joined.dtypes

transaction_type    object
transaction_hash    object
recipient           object
amountIn            object
amountOut           object
amountOutMin        object
amountInMax         object
payerIsUser         object
token0              object
fee                 object
token1              object
pool                object
dtype: object

In [16]:
engine = create_engine(postgresql_uri_mp)

df_joined.to_sql("swap_limit_price", engine, if_exists="append", index=False)

875